<a href="https://colab.research.google.com/github/Juethro/Llama3-fineTune-Chatbot-BukitVista/blob/main/Fine%20Tuning%20Llama-3%20%2B%20Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Library

In [ ]:
!pip install datasets -qqq

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

# Data Preparation

In [ ]:
from huggingface_hub import notebook_login

# Jalankan ini untuk login dan masukkan token Anda saat diminta
notebook_login()

In [ ]:
from datasets import load_dataset
dataset = load_dataset("Juethro01/sdcBaliUAS", split = "train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/353 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9992 [00:00<?, ? examples/s]

In [ ]:
dataset[1]

{'conversations': [{'from': 'human',
   'value': 'Sohuturon’s check-in time is: 3:00 PM - 5:00 PM'},
  {'from': 'human',
   'value': 'Hi. I will extend my stay untill the 9th of June thanks.'},
  {'from': 'human', 'value': 'Booking confirmed · 1 guest, Jun 7 – 9, 2019'},
  {'from': 'gpt',
   'value': 'Thank you for your extending.\n\nCheers,\nRafik & Bukit Vista Hosts'},
  {'from': 'human', 'value': 'Halo, apakah masih available? '},
  {'from': 'gpt',
   'value': 'Thank you for your interest in this charming accommodation. We would surely be delighted to host your upcoming holiday here in our little paradise! Glad to inform you that the room is available on your preferred traveling dates. Hereby I send you the pre-approval so you can continue to the next steps of reservation.\n\nRegards,\nTendi and Bukit Vista Hosts'},
  {'from': 'gpt',
   'value': 'Thank you for booking our accommodation, ! \n\nThe next step is to please go to guest.bukitvista.com to get clear information about the ad

# Fine Tuning

## Load Base Model (Llama-3)

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

## Create PEFT

In [ ]:
# Menggunakan model PEFT ()
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.1.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Converting Data into Chat Template

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/9992 [00:00<?, ? examples/s]

In [ ]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nhi\n\nWe are 3 spanish guys currently living in Singapore we will spend few days in uluwatu and going up to Seminiyak to end the trip. we believe your apartment will suit perfectly for us\nthanks<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nBooking confirmed · 3 guests, Jul 13 – 17, 2019<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nWelcome to MELIN INN, Ivan!\n\nAddress of MELIN INN:\nJalan Werkudara Gang Arjuna no 54 Legian\n\nGPS map of MELIN INN:\nhttp://bit.ly/bvmaps-melininn\n\nSend us your booking code on WhatsApp to keep in touch:\nhttps://bit.ly/chatmebv\n\n\nLooking forward to welcoming you very soon! 😊\n\nCheers,\nRafik, co-host of Bastian, Wayana and Bukit Vista Hosts<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThank you for booking our accommodation, Ivan!\n\nThe next step is to please go to guest.bukitvista.com/HMTXR9JDNH  to get clear information about the address and fill in 

In [ ]:
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'As an efficient assistant in customer service, you cater to guests under the name of GAIA (Guest AI Assistant). GAIA is dedicated to optimizing the efficiency and quality of your hospitality services, ensuring that every interaction with guests is elevated to excellence. Your role is to provide detailed answers to guests' inquiries, eliminating the need for them to seek information elsewhere. Remember, you represent both GAIA and Vista Hill Hosts in every interaction.\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

if False:
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,),
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
        map_eos_token = True,
    )

# Train Model

In [ ]:
# @title Setup SFFT Trainer
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/9992 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
7.246 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,992 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.583600
2,2.840900
3,2.435800
4,2.375500
5,2.493700
6,2.201000
7,2.373500
8,2.400700
9,2.266600
10,2.290300


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1064.7512 seconds used for training.
17.75 minutes used for training.
Peak reserved memory = 7.387 GB.
Peak reserved memory for training = 0.141 GB.
Peak reserved memory % of max memory = 50.088 %.
Peak reserved memory for training % of max memory = 0.956 %.


# Model Inference

In [ ]:
# Ubah ke mode inference
FastLanguageModel.for_inference(model)

# Prompt yang dimasukkan
messages = [
    {"from": "human",
     "value": """Title: Modern Cozy Room in Sunny Canggu
          Place: Private room in guesthouse, Kecamatan Kuta Utara, Indonesia

          Description:
          Escape to this guesthouse in Tibubeneng, offering 8 rooms for a memorable stay. Features include a fluffy double bed, private bathroom, air conditioning, 24 Mbps WiFi, shared pool, kitchen, and safety box. Ideal for long stays, it provides a tranquil sunbathing experience, scooter rental, and airport pickup. Enjoy privacy in your 16-square-meter room and the shared pool and kitchen. Nearby attractions include beaches, cafes, fitness centers, and markets. Premium services like spa, private tours, and romantic room decorations are available. Note: No breakfast, pet policy, and early check-in/late check-out terms apply.

          Title: Stunning 3BR Villa Near Padang Beach
          Place: Entire home in South Kuta, Indonesia

          Description:
          Perfect for long stays, this villa is close to Padang Beach and other Uluwatu beaches. It features 3 bedrooms with semi-outdoor bathrooms, a private pool, kitchen, living room, and spacious garden. Ideal for remote work with 150 Mbps WiFi, it's pet-friendly and offers easy access to beaches and restaurants. Guests have full access to the villa and pool. Note: No breakfast provided, local construction noise, and advice to secure belongings from local monkeys.



         Hi yesterday i went to the place but why when i arrive the bed was dirty and there's a cockroach running around, i was very dissapointed."""
    },
]

## Convert prompt menjadi chat template
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 2000, use_cache = True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Title: Modern Cozy Room in Sunny Canggu
          Place: Private room in guesthouse, Kecamatan Kuta Utara, Indonesia

          Description:
          Escape to this guesthouse in Tibubeneng, offering 8 rooms for a memorable stay. Features include a fluffy double bed, private bathroom, air conditioning, 24 Mbps WiFi, shared pool, kitchen, and safety box. Ideal for long stays, it provides a tranquil sunbathing experience, scooter rental, and airport pickup. Enjoy privacy in your 16-square-meter room and the shared pool and kitchen. Nearby attractions include beaches, cafes, fitness centers, and markets. Premium services like spa, private tours, and romantic room decorations are available. Note: No breakfast, pet policy, and early check-in/late check-out terms apply.

          Title: Stunning 3BR Villa Near Padang Beach
          Place: Entire home in South Kuta, Indonesia

          Description:
          Perfect for long stays

# Saving Model

In [ ]:
from google.colab import userdata

In [ ]:
# Save to q4_k_m GGUF
# model.push_to_hub_gguf("Juethro01/propertyCSChatBot", tokenizer, quantization_method = "q4_k_m", token = userdata.get('HUGGING_FACE')) # Huggingface save model]

# Just LoRA adapters
# model.push_to_hub_merged("Juethro01/propertyCSChatBotv1", tokenizer, save_method = "lora", token = userdata.get('HUGGING_FACE'))

# Merge to 16bit
model.push_to_hub_merged("Juethro01/propertyCSChatBotv2", tokenizer, save_method = "merged_16bit", token = userdata.get('HUGGING_FACE'))

Unsloth: You are pushing to hub, but you passed your HF username = Juethro01.
We shall truncate Juethro01/propertyCSChatBotv2 to propertyCSChatBotv2
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.83 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 38%|███▊      | 12/32 [00:01<00:02,  8.73it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:27<00:00,  2.74s/it]


Unsloth: Saving tokenizer...

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/Juethro01/propertyCSChatBotv2


# Load Model

## Load Model from HuggingFace using native unsloth (can be used for another fine tune)

In [ ]:
from unsloth import FastLanguageModel
from google.colab import userdata

# Define model parameters if needed
max_seq_length = 2048  # Adjust if necessary
dtype = None  # Adjust if necessary
load_in_4bit = True  # Adjust if necessary

# Load the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Juethro01/propertyCSChatBotv2",  # Your model name
    max_seq_length=max_seq_length,
    dtype=dtype,
    # load_in_4bit=load_in_4bit,
    # from_tf=True,
    # token=userdata.get('HUGGING_FACE'),  # Replace with your token if needed for private models, if it's public model then you can leave this to ""
    # device_map="auto", # Utilize GPU automatically if it's available
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

## Load Model from HuggingFace using Huggingface API (not recommended by Unsloth)

In [ ]:
if False: # kode dinonaktifkan | Code Deactivated

    # I highly do NOT suggest - use Unsloth if possible
    # Load model directly
    from transformers import AutoModel
    model = AutoModel.from_pretrained("Juethro01/propertyCSChatBotv1")

## Load GGUF Model using Llama_cpp (recommended for inference)

### There still problem with llama-cpp for python bindings (CUDA support)

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!python --version

Python 3.11.11


In [ ]:
# # CUDA support (problem: cannot install)
# !CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python

In [ ]:
# CPU only (worked)
!pip install --upgrade --quiet  llama-cpp-python

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
if True: # kode aktif | Code Activated
  from huggingface_hub import hf_hub_download
  from llama_cpp import Llama

  # Unduh model dari Hugging Face
  model_path = hf_hub_download(
      repo_id="Juethro01/propertyCSChatBot",  # Nama repositori di Hugging Face
      filename="unsloth.Q4_K_M.gguf"         # Nama file model di repositori
  )

  # Load model dari path lokal
  llm = Llama(
      model_path=model_path,  # Path ke file model yang diunduh
      n_gpu_layers=-1,        # Gunakan GPU untuk semua layer
      verbose=True            # Aktifkan log untuk debugging
  )

## Inference Model

### Hanya jika diload menggunakan unsloth

In [ ]:
# Ubah ke mode inference
FastLanguageModel.for_inference(model)

# Prompt yang dimasukkan
messages = [
    {"from": "human",
     "value": """Title: Modern Cozy Room in Sunny Canggu
      Place: Private room in guesthouse, Kecamatan Kuta Utara, Indonesia

      Description:
      Escape to this guesthouse in Tibubeneng, offering 8 rooms for a memorable stay. Features include a fluffy double bed, private bathroom, air conditioning, 24 Mbps WiFi, shared pool, kitchen, and safety box. Ideal for long stays, it provides a tranquil sunbathing experience, scooter rental, and airport pickup. Enjoy privacy in your 16-square-meter room and the shared pool and kitchen. Nearby attractions include beaches, cafes, fitness centers, and markets. Premium services like spa, private tours, and romantic room decorations are available. Note: No breakfast, pet policy, and early check-in/late check-out terms apply.

      Title: Stunning 3BR Villa Near Padang Beach
      Place: Entire home in South Kuta, Indonesia

      Description:
      Perfect for long stays, this villa is close to Padang Beach and other Uluwatu beaches. It features 3 bedrooms with semi-outdoor bathrooms, a private pool, kitchen, living room, and spacious garden. Ideal for remote work with 150 Mbps WiFi, it's pet-friendly and offers easy access to beaches and restaurants. Guests have full access to the villa and pool. Note: No breakfast provided, local construction noise, and advice to secure belongings from local monkeys.



      Hi can you give me recommendation a room like this one? give me to options."""
    },
]

## Convert prompt menjadi chat template
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 2000, use_cache = True)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Title: Modern Cozy Room in Sunny Canggu
      Place: Private room in guesthouse, Kecamatan Kuta Utara, Indonesia

      Description:
      Escape to this guesthouse in Tibubeneng, offering 8 rooms for a memorable stay. Features include a fluffy double bed, private bathroom, air conditioning, 24 Mbps WiFi, shared pool, kitchen, and safety box. Ideal for long stays, it provides a tranquil sunbathing experience, scooter rental, and airport pickup. Enjoy privacy in your 16-square-meter room and the shared pool and kitchen. Nearby attractions include beaches, cafes, fitness centers, and markets. Premium services like spa, private tours, and romantic room decorations are available. Note: No breakfast, pet policy, and early check-in/late check-out terms apply.

      Title: Stunning 3BR Villa Near Padang Beach
      Place: Entire home in South Kuta, Indonesia

      Description:
      Perfect for long stays, this villa is close to Pad

### Hanya jika diload menggunakan llama_cpp

In [ ]:
# Gabungkan prompt
prompt = """
Title: Modern Cozy Room in Sunny Canggu
Place: Private room in guesthouse, Kecamatan Kuta Utara, Indonesia

Description:
Escape to this guesthouse in Tibubeneng, offering 8 rooms for a memorable stay. Features include a fluffy double bed, private bathroom, air conditioning, 24 Mbps WiFi, shared pool, kitchen, and safety box. Ideal for long stays, it provides a tranquil sunbathing experience, scooter rental, and airport pickup. Enjoy privacy in your 16-square-meter room and the shared pool and kitchen. Nearby attractions include beaches, cafes, fitness centers, and markets. Premium services like spa, private tours, and romantic room decorations are available. Note: No breakfast, pet policy, and early check-in/late check-out terms apply.

Title: Stunning 3BR Villa Near Padang Beach
Place: Entire home in South Kuta, Indonesia

Description:
Perfect for long stays, this villa is close to Padang Beach and other Uluwatu beaches. It features 3 bedrooms with semi-outdoor bathrooms, a private pool, kitchen, living room, and spacious garden. Ideal for remote work with 150 Mbps WiFi, it's pet-friendly and offers easy access to beaches and restaurants. Guests have full access to the villa and pool. Note: No breakfast provided, local construction noise, and advice to secure belongings from local monkeys.

Hi yesterday I went to the place but why when I arrive the bed was dirty and there's a cockroach running around, even the bedsheet is stinks. I was very disappointed, This is the worst hospitality I ever experienced.

Response:
"""

# Inference dengan llama_cpp
response = llm(prompt, max_tokens=200, stop=["\n", "###"])
print(response["choices"][0]["text"])

# Evaluation Model

In [ ]:
!pip install deepeval -U -qq

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.1/585.1 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
u

In [ ]:
from google.colab import userdata
import os
# Get the API key from Colab secrets
openai_api_key = userdata.get('OPENAI_API_KEY')

# Set the environment variable
os.environ['OPENAI_API_KEY'] = openai_api_key

In [ ]:
from deepeval.metrics import GEval, BiasMetric, ToxicityMetric, HallucinationMetric
from deepeval.test_case import LLMTestCaseParams

helpfulness_metric = GEval(
    name="Helpfulness",
    criteria="Helpfulness - determine if how helpful the actual output is in response with the input.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    threshold=0.5
)
bias_metric = BiasMetric(threshold=0.5)
toxicity_metric = ToxicityMetric(threshold=0.5)

In [ ]:
from deepeval.test_case import LLMTestCase
from transformers import pipeline


inputs = [
    "If you had to recommend one Bukit Vista property to a friend, which property would you choose and why?",
    "Hello. We would like to extend our stay to tomorrow night please. Is this possible?",
    "Can you provide more details about the check-in process for our upcoming stay?",
    # "What amenities are included in the property we booked?",
    # "What is the cancellation policy for our reservation?"

]

pipe = pipeline(
  task="text-generation",
  model=model, # Loaded model from huggingface
  tokenizer=tokenizer,
  max_length=2000
)

test_cases = []
for input_text in inputs:
    messages = [
        {"from": "human", "value": input_text},
    ]
    inputs_tensor = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,  # Must add for generation
        return_tensors="pt",
    ).to("cuda")

    text_streamer = TextStreamer(tokenizer)
    generated_output = model.generate(input_ids=inputs_tensor, streamer=text_streamer, max_new_tokens=2000, use_cache=True)

    # Decode the generated tokens
    actual_output = tokenizer.decode(generated_output[0], skip_special_tokens=True)

    test_case = LLMTestCase(input=input_text, actual_output=actual_output)
    test_cases.append(test_case)

Device set to use cuda:0


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

If you had to recommend one Bukit Vista property to a friend, which property would you choose and why?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

What a great question! As a friendly AI, I don't have personal preferences or direct experiences, but I can provide some insights based on popular opinions and market trends.

Bukit Vista is a well-known property developer in Singapore, and they have a range of properties in various locations. However, if I had to recommend one property, I would suggest the Bukit Vista @ Holland Village.

Here's why:

1. **Prime location**: Holland Village is a popular neighborhood in the heart of Singapore, known for its vibrant atmosphere, eclectic mix of shops and restaurants, and proximity to the city center.
2. **Commuter-friendly**: The property is within walking distance to Holland MRT Station (Circle Line), making it easy for residents to commute to other parts of the city.
3. **A

In [ ]:
from deepeval.metrics import KnowledgeRetentionMetric
from deepeval.test_case import ConversationalTestCase


test_case = ConversationalTestCase(turns=test_cases)
metric = KnowledgeRetentionMetric(threshold=0.5)

metric.measure(test_case)
print(metric.score)
print(metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

1.0
The score is 1.00 because there are no attritions, indicating that the LLM consistently retained and correctly referenced all previously established knowledge in the conversation.


In [ ]:
from deepeval.dataset import EvaluationDataset

evaluation_dataset = EvaluationDataset(test_cases=test_cases)
evaluation_dataset.evaluate([bias_metric, helpfulness_metric, toxicity_metric])

✨ You're running DeepEval's latest Bias Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Helpfulness (GEval) Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Toxicity Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 3 test case(s) in parallel: |██████████|100% (3/3) [Time Taken: 00:08,  2.68s/test case]



Metrics Summary

  - ✅ Bias (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.00 because the actual output is exemplary, demonstrating a high level of fairness and objectivity with no identifiable bias. The phrasing and arguments are balanced and inclusive, showcasing a thorough understanding of diverse perspectives., error: None)
  - ✅ Helpfulness (GEval) (score: 0.669505850770532, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The response acknowledges the request and asks for necessary information to provide a precise answer, which partially addresses the problem but fails to directly offer guidance or anticipate follow-up questions., error: None)
  - ✅ Toxicity (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.00 because the actual output is entirely non-toxic and promotes positivity without any harmful language., error: None)

For test case:

  - input: Can you provide more de

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

EvaluationResult(test_results=[TestResult(name='test_case_2', success=True, metrics_data=[MetricData(name='Bias', threshold=0.5, success=True, score=0.0, reason='The score is 0.00 because the actual output is exemplary, demonstrating a high level of fairness and objectivity with no identifiable bias. The phrasing and arguments are balanced and inclusive, showcasing a thorough understanding of diverse perspectives.', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.002045, verbose_logs='Opinions:\n[] \n \nVerdicts:\n[]'), MetricData(name='Helpfulness (GEval)', threshold=0.5, success=True, score=0.669505850770532, reason='The response acknowledges the request and asks for necessary information to provide a precise answer, which partially addresses the problem but fails to directly offer guidance or anticipate follow-up questions.', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.0026449999999999998, verbose_logs='Criteria:\nHelpfulne